# Entrenamiento del modelo

In [3]:
import torch
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os

# Definir transformaciones
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(299),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(299),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'directorio_datos'
batch_size = 32

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4)
               for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes


In [ ]:
from torch import nn

model = models.inception_v3(pretrained=True)
# Ajustar la última capa para 5 clases
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 5)
from torch import optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
num_epochs = 12

for epoch in range(num_epochs):
    # Inicializa variables para almacenar resultados tanto de entrenamiento como de validación
    print(f'Epoch {epoch+1}/{num_epochs}')
    
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # Pone el modelo en modo de entrenamiento
        else:
            model.eval()   # Pone el modelo en modo de evaluación
        
        running_loss = 0.0
        running_corrects = 0

        # Itera sobre los datos.
        for inputs, labels in dataloaders[phase]:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            # Solo sigue el historial si está en fase de 'train'
            with torch.set_grad_enabled(phase == 'train'):
                # Inception v3 devuelve outputs en modo entrenamiento como una tupla
                if phase == 'train':
                    outputs, aux_outputs = model(inputs)
                    loss1 = criterion(outputs, labels)
                    loss2 = criterion(aux_outputs, labels)
                    loss = loss1 + 0.4*loss2
                else:
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                _, preds = torch.max(outputs, 1)

                # backward + optimize solo si está en fase de 'train'
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f'{phase} Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}')

# Guardado del modelo

In [8]:
torch.save(model.state_dict(), 'inception_v3_model.pth')


# Matriz de confusión y recall

In [ ]:
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import confusion_matrix

model.eval()

# Define las transformaciones
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dispositivo (GPU o CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# Ruta al conjunto de datos
data_dir = 'directorio_test'

# Cargar el conjunto de datos
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=4)
class_names = dataset.classes

y_pred = []
y_true = []

# Desactivar el cálculo de gradientes
with torch.no_grad():
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        y_pred.extend(predicted.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

# Calcular la matriz de confusión
confusion_mat = confusion_matrix(y_true, y_pred)
recall = recall_score(y_true, y_pred, average='macro') 
print(confusion_mat)
print(f'Recall del modelo: {recall}')
